# Mapping the world's coast: A global 100-m coastal typology derived from satellite data using deep learning

This notebook provides a brief introduction and access routines for the **Global Coastal Typology** dataset, available through the Global Coastal Transect Repository ([Calkoen et al., 2025](https://www.sciencedirect.com/science/article/pii/S1364815224003189)). Data is currently accessible with an access token and will be made publicly available upon publication.

The coastal typology is a globally consistent, 100-m resolution classification derived from satellite imagery and elevation data using deep learning. It provides four attributes along nearly 10 million coastal transects: sediment type, coastal type, built environment presence, and coastal defense presence. The model was trained on ~1800 globally distributed samples and shows strong predictive performance, with F1 scores ranging from 0.67 to 0.83.

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.796, 53.108, 5.229, 53.272))
west, south, east, north = roi.geometry.item().bounds

## Fetch data from the database

In [ ]:
import coastpy

COLUMNS = [
    "transect_id",
    "lon",
    "lat",
    "geometry",
    "utm_epsg",
    "bbox",
    "quadkey",
    "continent",
    "country",
    "common_region_name",
    "class:shore_type",
    "class:coastal_type",
    "class:has_defense",
    "class:is_built_environment",
]
db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    # Use columns to filter when you don't need all data;
    columns=COLUMNS,
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
df = df.assign(geometry=gpd.GeoSeries.from_xy(df.lon, df.lat, 4326))
print(f"Shape: {df.shape}")
df.head()

In [ ]:
import importlib

import coastpy.typology.encoding
import coastpy.typology.visualization

importlib.reload(coastpy.typology.visualization)
importlib.reload(coastpy.typology.encoding)

from coastpy.typology.visualization import CoastalTypologyDashboard

In [ ]:
CoastalTypologyDashboard(df).show()